In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse

# 로그인 정보 하드코딩
USERNAME = 'loextasy@naver.com'  # 여기에 Instagram 사용자 이름을 입력하세요
PASSWORD = 'v912803V!!!'  # 여기에 Instagram 비밀번호를 입력하세요

def setup_driver():
    """Chrome 드라이버를 설정하고 반환합니다."""
    chrome_driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
    service = Service(chrome_driver_path)

    options = Options()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=service, options=options)
    return driver

def login_instagram(driver):
    """Instagram에 로그인합니다."""
    driver.get('https://www.instagram.com/accounts/login/')
    time.sleep(2)

    username_input = driver.find_element(By.NAME, 'username')
    password_input = driver.find_element(By.NAME, 'password')
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')

    username_input.send_keys(USERNAME)
    password_input.send_keys(PASSWORD)
    login_button.click()

    time.sleep(5)  # 로그인 후 대기

def crawl_instagram(driver, search_query):
    """주어진 검색어로 Instagram에서 데이터를 크롤링합니다."""
    search_query_encoded = urllib.parse.quote(search_query)
    search_url = f'https://www.instagram.com/explore/tags/{search_query_encoded}/'
    
    driver.get(search_url)
    time.sleep(30)  # 페이지가 완전히 로드될 때까지 대기

    SCROLL_PAUSE_TIME = 1
    alt_texts = []

    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        pagestring = driver.page_source
        bs = BeautifulSoup(pagestring, 'lxml')
        
        post_divs = bs.find_all('div', {'class': '_aagv'})
        
        for div in post_divs:
            img_tags = div.find_all('img')
            for img in img_tags:
                alt_text = img.get('alt')
                if alt_text:
                    alt_texts.append(alt_text)
        
        if post_divs:
            print(f"Found {len(post_divs)} divs with class '_aagv' on this scroll.")
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == last_height:
            break
        last_height = new_height

    return alt_texts

def save_to_excel(data_dict, current_time):
    """크롤링한 데이터를 엑셀 파일로 저장합니다."""
    all_data = []
    for search_query, alt_texts in data_dict.items():
        if alt_texts:
            df = pd.DataFrame({
                "검색어": [search_query] * len(alt_texts),
                "검색결과": alt_texts
            })
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"인스타그램크롤링_{current_time}.xlsx", index=False)
        print("크롤링 데이터가 엑셀 파일로 저장되었습니다.")
    else:
        print("No data found.")

def main():
    current_time = datetime.now().strftime('%y-%m-%d')
    
    # 검색어 리스트 입력
    search_queries = [
    # 발달상권
    "성수카페거리",
    "가로수길",
    "북촌한옥마을",
    "인사동·익선동",
    "광장(전통)시장",
    "압구정로데오거리",
    
    # 인구밀집지역
    "홍대입구역(2호선)",
    "이태원역",
    "강남역",
    "서울식물원·마곡나루역",
    
    # 관광특구
    "동대문 관광특구",
    "명동 관광특구",
    "잠실 관광특구",
    "종로·청계 관광특구",
    
    # 고궁 문화유산
    "경복궁",
    "광화문·덕수궁",
    "창덕궁 종묘",
    "보신각",
    
    # 공원
    "반포한강공원",
    "고척돔",
    "남산공원",
    "잠실종합운동장",
    "서울숲공원",
    "국립중앙박물관·용산가족공원",
    "서울대공원"
    ]

    driver = setup_driver()
    login_instagram(driver)
    
    data_dict = {}
    for search_query in search_queries:
        alt_texts = crawl_instagram(driver, search_query)
        data_dict[search_query] = alt_texts
    
    save_to_excel(data_dict, current_time)
    
    driver.quit()

if __name__ == "__main__":
    main()

Found 27 divs with class '_aagv' on this scroll.
Found 28 divs with class '_aagv' on this scroll.
Found 30 divs with class '_aagv' on this scroll.
크롤링 데이터가 엑셀 파일로 저장되었습니다.


In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib.parse
import re

# 로그인 정보 하드코딩
USERNAME = 'loextasy@naver.com'  # 여기에 Instagram 사용자 이름을 입력하세요
PASSWORD = 'v912803V!!!'  # 여기에 Instagram 비밀번호를 입력하세요

def setup_driver():
    """Chrome 드라이버를 설정하고 반환합니다."""
    chrome_driver_path = 'C:\\chromedriver-win64\\chromedriver.exe'
    service = Service(chrome_driver_path)

    options = Options()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")

    driver = webdriver.Chrome(service=service, options=options)
    return driver

def login_instagram(driver):
    """Instagram에 로그인합니다."""
    driver.get('https://www.instagram.com/accounts/login/')
    time.sleep(2)

    username_input = driver.find_element(By.NAME, 'username')
    password_input = driver.find_element(By.NAME, 'password')
    login_button = driver.find_element(By.XPATH, '//button[@type="submit"]')

    username_input.send_keys(USERNAME)
    password_input.send_keys(PASSWORD)
    login_button.click()

    time.sleep(5)  # 로그인 후 대기

def remove_parentheses(query):
    """검색어에서 괄호와 괄호 안의 내용을 제거합니다."""
    return re.sub(r'\(.*?\)', '', query).strip()

def crawl_instagram(driver, search_query):
    """주어진 검색어로 Instagram에서 데이터를 크롤링합니다."""
    search_query_encoded = urllib.parse.quote(remove_parentheses(search_query).replace(' ', ''))
    search_url = f'https://www.instagram.com/explore/tags/{search_query_encoded}/'
    
    driver.get(search_url)
    time.sleep(30)  # 페이지가 완전히 로드될 때까지 대기

    SCROLL_PAUSE_TIME = 1
    alt_texts = []

    last_height = driver.execute_script('return document.body.scrollHeight')
    while True:
        pagestring = driver.page_source
        bs = BeautifulSoup(pagestring, 'lxml')
        
        post_divs = bs.find_all('div', {'class': '_aagv'})
        
        for div in post_divs:
            img_tags = div.find_all('img')
            for img in img_tags:
                alt_text = img.get('alt')
                if alt_text:
                    alt_texts.append(alt_text)
        
        if post_divs:
            print(f"Found {len(post_divs)} divs with class '_aagv' on this scroll.")
        
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script('return document.body.scrollHeight')
        
        if new_height == last_height:
            break
        last_height = new_height

    return alt_texts

def save_to_excel(data_dict, current_time):
    """크롤링한 데이터를 엑셀 파일로 저장합니다."""
    all_data = []
    for search_query, alt_texts in data_dict.items():
        if alt_texts:
            df = pd.DataFrame({
                "검색어": [search_query] * len(alt_texts),
                "검색결과": alt_texts
            })
            all_data.append(df)

    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        final_df.to_excel(f"인스타그램크롤링_{current_time}.xlsx", index=False)
        print("크롤링 데이터가 엑셀 파일로 저장되었습니다.")
    else:
        print("No data found.")

def main():
    current_time = datetime.now().strftime('%y-%m-%d')
    
    # 검색어 리스트 입력
    search_queries = [
        # 발달상권
        "성수카페거리",
        "가로수길",
        "북촌한옥마을",
        "인사동·익선동",
        "광장(전통)시장",
        "압구정로데오거리",
        
        # 인구밀집지역
        "홍대입구역(2호선)",
        "이태원역",
        "강남역",
        "서울식물원·마곡나루역",
        
        # 관광특구
        "동대문 관광특구",
        "명동 관광특구",
        "잠실 관광특구",
        "종로·청계 관광특구",
        
        # 고궁 문화유산
        "경복궁",
        "광화문·덕수궁",
        "창덕궁·종묘",
        "보신각",
        
        # 공원
        "반포한강공원",
        "고척돔",
        "남산공원",
        "잠실종합운동장",
        "서울숲공원",
        "국립중앙박물관·용산가족공원",
        "서울대공원"
    ]

    driver = setup_driver()
    login_instagram(driver)
    
    data_dict = {}
    for search_query in search_queries:
        # 검색어에 '·'가 포함된 경우, 이를 기준으로 분할
        if '·' in search_query:
            sub_queries = search_query.split('·')
            combined_results = []
            for sub_query in sub_queries:
                alt_texts = crawl_instagram(driver, sub_query.strip())
                combined_results.extend(alt_texts)
            data_dict[search_query] = combined_results
        else:
            alt_texts = crawl_instagram(driver, search_query)
            data_dict[search_query] = alt_texts
    
    save_to_excel(data_dict, current_time)
    
    driver.quit()

if __name__ == "__main__":
    main()

Found 4 divs with class '_aagv' on this scroll.
Found 7 divs with class '_aagv' on this scroll.
Found 28 divs with class '_aagv' on this scroll.
